<a href="https://colab.research.google.com/github/edytaa/FaceRaterForGANs/blob/dev%2Fmultiple_generations/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

fatal: destination path 'stylegan2' already exists and is not an empty directory.


In [ ]:
!mkdir /content/stimuli/
!mkdir /content/stimuli/images/
!mkdir /content/stimuli/latent/

mkdir: cannot create directory ‘/content/stimuli/’: File exists
mkdir: cannot create directory ‘/content/stimuli/images/’: File exists
mkdir: cannot create directory ‘/content/stimuli/latent/’: File exists


In [ ]:
%tensorflow_version 1.x
import tensorflow
print('Tensorflow version: {}'.format(tensorflow.__version__) )

TensorFlow 1.x selected.
Tensorflow version: 1.15.2


In [ ]:
import sys
import argparse
import numpy as np
import PIL.Image
import re
import cv2
import random
from scipy.io import savemat
from sklearn.metrics.pairwise import euclidean_distances

path_stylegan = r'/content/stylegan2'
sys.path.append(path_stylegan)

import dnnlib
import dnnlib.tflib as tflib
import pretrained_networks
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from IPython.display import display, clear_output, Image

In [ ]:
network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
truncation_psi = 0.5 #threshold that is used to truncate and resample the latent vectors that are above the threshold

# Return 3 networks, but we will be mainly using Gs
# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

# Get tf noise variables, for the stochastic variation
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.


In [ ]:
Gs_kwargs = dnnlib.EasyDict()
Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_kwargs.randomize_noise = False
if truncation_psi is not None:
    Gs_kwargs.truncation_psi = truncation_psi

In [90]:
class RateGeneration:
  def __init__(self, imPth_, nTrl_, generation_, 
              nSurv_, nRnd_, wFitterParent_, 
                    basePth_, allZ, noise_vars_, Gs_, Gs_kwargs_, nInPool, mutAmp, mutP):
      self.button = widgets.Button(description="Start!")
      self.output = widgets.Output()
      self.slider = widgets.IntSlider(value=5, max=9)
      self.button.on_click(self.on_button_clicked)
      self.nTrl = nTrl_
      self.imPath = imPth_
      self.basePth_ = basePth_
      self.i = 0
      self.generation = generation_
      self.responses = []
      self.nSurv_ = nSurv_
      self.nRnd_ = nRnd_
      self.wFitterParent_ = wFitterParent_
      self.allZ = allZ
      self.noise_vars_ = noise_vars_
      self.Gs_ = Gs_
      self.Gs_kwargs_ = Gs_kwargs_
      self.nInPool = nInPool
      self.mutP = mutP
      self.mutAmp = mutAmp
      self.genObsImages()
      display(self.button, self.slider, self.output)
    
  def on_button_clicked(self, b):
      if self.i <= self.nTrl:
        b.description = "Next"
        self.show(self.i != 0)

      if self.i > self.nTrl:  # all images have been shown in this generation -> show rating and generate new images
        b.disabled = True
        self.slider.disabled = True
        with self.output:
          print(f"Rating of generation {self.generation} finished")
          print('Start generating new images...... please wait......')
          self.evaluateOneGeneration() 
          self.genObsImages()
          print('New images generated')
          self.generation += 1
          print(f'Starting new generation (gen: {self.generation})')
        self.i = 0
        self.responses = []
        b.description = "Start next gen"
        self.slider.disabled = False
        b.disabled = False

    
  def show(self, get_grading=False): # get grade and show next image
      with self.output:
        print(f"Button pressed. Generation {self.generation}. \
          Picture nr {self.i}. Last grade = {self.slider.value}")
        display(Image(f"{self.imPath}trl_{self.i}.png"))
        if get_grading:
          self.responses.append(self.slider.value)
        clear_output(wait=True)
      self.i += 1

  def evaluateOneGeneration(self):
    
    thsResp = self.responses

    # save current latents and responses
    thsLatentPth = self.basePth_ + 'stimuli/latent/generation_'+str(self.generation)+'.mat'
    savemat(thsLatentPth, {"allZ": self.allZ, "thsResp": thsResp})
          
    # transform responses to probabilities (needed for sampling parents)
    thsFitness = softmax(thsResp)
          
    # take latent vectors of nSurvival highest responses
    thsIndices = np.argsort(thsFitness)
    thsSurv = self.allZ[thsIndices[-self.nSurv_:],:]
          
    # generate recombinations from 2 parent latent vectors of current gen 
    # w. fitness proportional to probability of being parent
    # parent with higher fitness contributes more
    thsPool = np.zeros([self.nInPool,self.allZ.shape[1]])

    for rr in range(self.nInPool):
      thsParents = np.random.choice(self.nTrl, 2, False, thsFitness)
      
      if thsFitness[thsParents[0]] > thsFitness[thsParents[1]]:
        contrib0 = self.wFitterParent_
        contrib1 = 1 - self.wFitterParent_
      elif thsFitness[thsParents[0]] < thsFitness[thsParents[1]]:
        contrib0 = 1 - self.wFitterParent_
        contrib1 = self.wFitterParent_
      elif thsFitness[thsParents[0]] == thsFitness[thsParents[1]]:
        contrib0 = .5   
        contrib1 = .5
              
      thsPool[rr,:] = self.allZ[thsParents[0],:] * contrib0 + self.allZ[thsParents[1],:] * contrib1

    # each latent dimension of children in recombined pool has some probability of mutation
    toEdit = np.random.choice([0, 1], (self.nInPool,thsPool.shape[1]), True, [1-self.mutP, self.mutP]) # mutP global
    thsEdits = np.random.randn(np.sum(toEdit)) * self.mutAmp # mutAmp global 
    thsPool[np.nonzero(toEdit)] = thsPool[np.nonzero(toEdit)] + thsEdits
          
    # add some random faces to the mix
    thsRnd = np.random.randn(self.nRnd_, 512)
      
    # combine direct survivals and recombined / mutated pool
    self.allZ = np.concatenate((thsSurv, thsPool, thsRnd),axis=0)
    # shuffle order of trials
    np.random.shuffle(self.allZ)

  def genObsImages(self):
      for tt in range(self.allZ.shape[0]):
          thsTrlPth = self.imPath+'trl_'+str(tt)+'.png'
          print('Generating image for trial %d ...' % tt)
          z = self.allZ[np.newaxis,tt,:]
          tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in self.noise_vars_}) # [height, width]
          images = self.Gs_.run(z, None, **self.Gs_kwargs_) # [minibatch, height, width, channel]
          PIL.Image.fromarray(images[0], 'RGB').save(thsTrlPth)
  

In [42]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
# genetic algorithm
nGen = 2
nTrl = 3
nSurv = 1 # number of best samples that survives the grading process (and prob goes to next generation)
nRnd = 1
nInPool = nTrl-nSurv-nRnd
wFitterParent = .75
mutAmp = .4
mutP = .3

In [ ]:
basePth = '/content/'
imPth = basePth + 'stimuli/images/'
seed = 1
rnd = np.random.RandomState(seed)

In [91]:
allZ = np.random.randn(nTrl,512)
rate_gen = RateGeneration(imPth, nTrl, 0, nSurv, nRnd, wFitterParent, 
                    basePth, allZ, noise_vars, Gs, Gs_kwargs, nInPool, mutAmp, mutP)

Generating image for trial 0 ...
Generating image for trial 1 ...
Generating image for trial 2 ...


Button(description='Start!', style=ButtonStyle())

IntSlider(value=5, max=9)

Output()

Generation 0
Generating image for trial 0 ...
Generating image for trial 1 ...
Generating image for trial 2 ...


Button(description='Start!', style=ButtonStyle())

IntSlider(value=5, max=9)

Output()

# New section